# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [27]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import numpy as np
import os
import json
import folium

# Import API key
from api_keys import geoapify_key

In [28]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,-25.65,89,98,25.79,TF,1681785819
1,1,port mathurin,-19.6833,63.4167,-19.57,74,11,18.10,MU,1681785819
2,2,afaahiti,-17.7500,-149.2833,-18.73,78,98,11.56,PF,1681785936
3,3,grantsville,40.5999,-112.4644,-21.84,31,100,9.22,US,1681785819
4,4,mogadishu,2.0371,45.3438,-18.94,70,98,6.91,SO,1681785717


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [29]:
%%capture --no-display

# Configure the map plot
import folium

# Create a map centered on a specific location
humidy_map = folium.Map(location=[40, -100], zoom_start=4)

# Add a point for each city in the DataFrame
for index, row in city_data_df.iterrows():
    # Get the latitude, longitude, and humidity for the city
    lat = row['Lat']
    lon = row['Lng']
    humidity = row['Humidity']
    
    # Create a marker with the size based on the humidity
    marker = folium.CircleMarker(location=[lat, lon], radius=humidity/10, color='red', fill=True, fill_color='red')
    
    # Add the marker to the map
    marker.add_to(humidy_map)
    
#Save Map
humidy_map.save('humidity_map.html')    

# Display the map
humidy_map


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [30]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[(city_data_df['Max Temp'] >= 21) & (city_data_df['Max Temp'] <= 27) & (city_data_df['Wind Speed'] < 10) & (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head(15)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


### Step 3: Create a new DataFrame called `hotel_df`.

In [25]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = np.empty(len(hotel_df), dtype=str)

# Display sample data
hotel_df.head(15)

,City,Country,Lat,Lng,Humidity,Hotel Name
27,ghat,LY,24.9647,10.1728,19,
55,arraial do cabo,BR,-22.9661,-42.0278,94,
91,ocala,US,29.1872,-82.1401,35,
169,port saint john's,ZA,-31.6229,29.5448,38,
185,karbala,IQ,32.6160,44.0249,32,
207,abu dhabi,AE,24.4667,54.3667,66,
259,malvan,IN,16.0667,73.4667,81,
286,kattivakkam,IN,13.2167,80.3167,92,
290,dwarka,IN,22.2394,68.9678,82,
291,port shepstone,ZA,-30.7414,30.4550,42,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [31]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    json_data = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
    
        #print(json_data)    
        hotel_df.loc[index, "Hotel Name"] = json_data["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
ghat - nearest hotel: فندق تاسيلي
arraial do cabo - nearest hotel: No hotel found
ocala - nearest hotel: Hilton Garden Inn
port saint john's - nearest hotel: Outback Inn
karbala - nearest hotel: فندق القمر
abu dhabi - nearest hotel: Marriott Downtown Abu-Dhabi
malvan - nearest hotel: Hotel Silver Sand
kattivakkam - nearest hotel: No hotel found
dwarka - nearest hotel: The Dwarika Hotel
port shepstone - nearest hotel: The Spot Backpackers'
east london - nearest hotel: No hotel found
longboat key - nearest hotel: Zota Beach Resort
toliara - nearest hotel: Ambary
west melbourne - nearest hotel: Courtyard by Marriott
al ghayzah - nearest hotel: فندق تاج العرب
tomatlan - nearest hotel: Hotel Hacienda Vieja Tomatlan


,City,Country,Lat,Lng,Humidity,Hotel Name
27,ghat,LY,24.9647,10.1728,19,فندق تاسيلي
55,arraial do cabo,BR,-22.9661,-42.0278,94,No hotel found
91,ocala,US,29.1872,-82.1401,35,Hilton Garden Inn
169,port saint john's,ZA,-31.6229,29.5448,38,Outback Inn
185,karbala,IQ,32.6160,44.0249,32,فندق القمر
207,abu dhabi,AE,24.4667,54.3667,66,Marriott Downtown Abu-Dhabi
259,malvan,IN,16.0667,73.4667,81,Hotel Silver Sand
286,kattivakkam,IN,13.2167,80.3167,92,No hotel found
290,dwarka,IN,22.2394,68.9678,82,The Dwarika Hotel
291,port shepstone,ZA,-30.7414,30.4550,42,The Spot Backpackers'


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [34]:
%%capture --no-display

# Configure the map plot

# Create a map object
hotel_map = folium.Map(location=[0, 0], zoom_start=5)

# Define a function to create the tooltip text for each marker
def create_tooltip(row):
    return f"{row['Hotel Name']}, {row['Country']}"

# Add markers to the map for each hotel in the DataFrame
for index, row in hotel_df.iterrows():
    folium.Marker(location=[row['Lat'], row['Lng']],
                  tooltip=create_tooltip(row),
                  icon=folium.Icon(color='blue')
                 ).add_to(hotel_map)



#Save Map
hotel_map.save('hotel_map.html')   

# Display the map
hotel_map
